## Utilisation API Pôle emploi
Source : https://github.com/etiennekintzler/api-offres-emploi

In [48]:
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
from requests.exceptions import HTTPError

import datetime
import time
import re
import pandas as pd 

In [49]:
client_id="PAR_textmining_6361ebc3d9de749cdfb72158572fa975a0ba30e97e4e570957f187132bb2b361"
client_secret="d77c395663a600cb89716cd8a6c02ab9b2d6920fbdc520791bcff93578b2f093"

In [50]:
client_id="PAR_test_085218c044f273f3c6642fcf3a5019ec4c4f6f5c0388dc65141609db5ddf2b12"
client_secret="b903c0540ecb05b669cf5b1fe8d640ee42e46a1b340841a00d7e0656d41abaa1"

In [51]:
def get_token(client_id,client_secret):
    data = dict(
                grant_type="client_credentials",
                client_id=client_id,
                client_secret=client_secret,
                scope="api_offresdemploiv2 o2dsoffre application_{}".format(client_id),
            )

    headers = {"content-type": "application/x-www-form-urlencoded"}
    params = dict(realm="/partenaire")
    current_time = datetime.datetime.today()
    r = requests.post(
        url="https://entreprise.pole-emploi.fr/connexion/oauth2/access_token",
        headers=headers,
        data=data,
        params=params,
        timeout=60
            )
    try:
        r.raise_for_status()
    except HTTPError as error:
        if r.status_code == 400:
            complete_message = str(error) + "\n" + str(r.json())
            raise HTTPError(complete_message)
        else:
            raise error
    else:
        token = r.json()
        token["expires_at"] = current_time + datetime.timedelta(
            seconds=token["expires_in"]
        )
        
        return token

In [52]:
token=get_token(client_id,client_secret)
token

{'access_token': 'OIPvRNzwq7Vm0xyWUuh17Fq-os0',
 'scope': 'application_PAR_test_085218c044f273f3c6642fcf3a5019ec4c4f6f5c0388dc65141609db5ddf2b12 api_offresdemploiv2 o2dsoffre',
 'token_type': 'Bearer',
 'expires_in': 1499,
 'expires_at': datetime.datetime(2024, 1, 11, 19, 20, 43, 502511)}

In [53]:
def get_hearders(token):
    headers = {
            "Authorization": "Bearer {}".format(token["access_token"])
        }
    return headers


In [54]:
# Valeurs constantes

OFFRES_DEMPLOI_V2_BASE = "https://api.emploi-store.fr/partenaire/offresdemploi/v2"
REFERENTIEL_ENDPOINT = "{}/referentiel".format(OFFRES_DEMPLOI_V2_BASE)
SEARCH_ENDPOINT = "{}/offres/search".format(OFFRES_DEMPLOI_V2_BASE)

In [55]:
# 'Réferentiel' available: domaine, appellations (domaines professionnelles ROME), metiers, themes, continents,
#         pays, regions, departements , communes , secteursActivites, naturesContrats,  typesContrats, niveauxFormations,
#         permis, langues

def referentiel(ref):
    
    r = requests.Session().get(
                url="{}/{}".format(REFERENTIEL_ENDPOINT, ref),
                params= dict(realm="/partenaire"),
                headers=get_hearders()
            )

    try:
        r.raise_for_status()
    except Exception as e:
        raise e
    else:
        return r.json()

In [56]:
def search(params=None):
    print('Requête avec les params {}'.format(params))
    r = requests.Session().get(
            url=SEARCH_ENDPOINT,
            params=params,
            headers=get_hearders(token)
        )

    silent_http_errors=False
    try:
        r.raise_for_status()
    except HTTPError as error:
        if r.status_code == 400:
            complete_message = str(error) + "\n" + r.json()["message"]
            if silent_http_errors:
                print(complete_message)
            else:
                raise HTTPError(complete_message)
        else:
            if silent_http_errors:
                print(str(error))
            else:
                raise error
    else:
        found_range = re.search(
            pattern="offres (?P<first_index>\d+)-(?P<last_index>\d+)/(?P<max_results>\d+)",
            string=r.headers["Content-Range"],
        ).groupdict()
        out = r.json()
        out.update({"Content-Range": found_range})
        return out

<>:26: SyntaxWarning: invalid escape sequence '\d'
<>:26: SyntaxWarning: invalid escape sequence '\d'
C:\Users\barai\AppData\Local\Temp\ipykernel_1564\1347239471.py:26: SyntaxWarning: invalid escape sequence '\d'
  pattern="offres (?P<first_index>\d+)-(?P<last_index>\d+)/(?P<max_results>\d+)",


### Extraction

In [57]:
start_dt = datetime.datetime(2020, 3, 1, 12, 30)
end_dt = datetime.datetime.today()
params = {

    "motsCles": "data",
    'minCreationDate': start_dt.strftime("%Y-%m-%dT%H:%M:%SZ"),
    'maxCreationDate': end_dt.strftime("%Y-%m-%dT%H:%M:%SZ")
}

ex_search=search(params)


Requête avec les params {'motsCles': 'data', 'minCreationDate': '2020-03-01T12:30:00Z', 'maxCreationDate': '2024-01-11T18:55:45Z'}


In [58]:
ex_search

{'resultats': [{'id': '167PCTC',
   'intitule': 'Ingénieur Data modeler et/ou TechData - (H/F)',
   'description': 'Nous recrutons un.e Ingénieur Data Modeler et/ou Tech Data pour rejoindre notre Business Unit INDUSTRIE au sein de la Business Line Data Intelligence. Elle accompagne nos clients dans leurs problématiques associées à la transformation digitale. Nos offres se déclinent autour de la data intelligence & de la maintenance prédictive.\n\nLe client, en tant que service responsable de la définition et la gouvernance des données, a pour mission globale de spécifier et modéliser les données techniques et leurs affichages. Il est aussi en charge des processus et de l\'exécution les opérations de gouvernance des données. Cette mission est fortement liée à la stratégie de digitalisation du client.\n\nDans le cadre de la mission, vous serez intégré.e à l\'équipe Tech data / Modélisation de données afin de :\n\n- Analyser les besoins métiers ;\n- Décrire et modéliser les objets de donn

# WEBSCRAPING APEC

##  Fonctions

In [59]:
import json

# pour exploiter les requêtes
from requests import post

# pour le contrôle des requêtes
from time import sleep, time
from random import randint
from abc import ABCMeta, abstractmethod
from warnings import warn



def convert_arr_2_string(arr, sep):
    """ Convert array to string with separator """
    return sep.join(arr)

def get_term(path):
    """ get term in a path. Otherwise, return 'Non renseigné' """
    if path is not None:
        return path.text.strip()
    return ''

def jprint(obj):
    """ convert array to json """
    # create a formatted string of the Python JSON object
    return json.dumps(obj, sort_keys=True) #, indent=4 

def post_data(root_path, payload, requests, start_time):
    """ post data and get the result  """
    response = post(root_path, json=payload)
    content = response.content
    
    ### pause de 8 à 15s
    sleep(randint(8, 15))
    
    ### afficher les informations sur les requêtes
    requests += 1 # incrémentation du nombre de requête
    elapsed_time = time() - start_time
    
    ### avertir si le code status est différent de 200
    if response.status_code != 200:
        warn('Request: {}; Status code:{}'.format(requests, requests/elapsed_time))
    
    ### stopper quand les requêtes atteignent le quota
    if requests > 200:
        warn('Nombre de requêtes trop important')
        return
    
    try:
        json_data = json.loads(content)
    except:
        json_data = ""
    
    return json_data


class scraping_jobs(metaclass=ABCMeta):
    
    def __init__(self, s_job, type_contract):
        self.s_job = s_job
        self.type_contract = type_contract
    
    @abstractmethod
    def scrap_job(self, dict_jobs, s_job, type_contract):
        pass


class scraping_jobs_apec(scraping_jobs):
    
    #
    def set_code_dpt(self, code_dpt):
        self.code_dpt = code_dpt
    
    #
    def scrap_job(self):
        ### paramètres pris
        param_search_words = self.s_job 
        
        ### pages à parcourir
        pages = [str(i) for i in range(0, 3)]  #### le nombre de pages voulu 
        requests = 0
        start_time = time()
        
        dict_jobs = []
        
        ### parcours des pages
        for page in pages:
            #
            root_path = 'https://www.apec.fr/cms/webservices/rechercheOffre'
            payload = {
                'typeClient': 'CADRE',
                'sorts' : [{
                    'type': 'SCORE',
                    'direction': 'DESCENDING'
                }],
                'pagination': {
                    'range': 20,
                    'startIndex': page
                },
                'activeFiltre': True,
                'pointGeolocDeReference': {
                    'distance': 0
                },
                'motsCles': param_search_words
            }
            
            json_data = post_data(root_path, payload, requests, start_time)
            
            ### vérifier l'existence de l'index 'resultats'
            if 'resultats' in json_data:
                result_containers = json_data['resultats']
            
                ### extraction des données du JSON renvoyé
                for result in result_containers:
                    #
                    if result['localisable'] == True:
                        latitude = result['latitude'],
                        longitude = result['longitude'],
                    else:
                        latitude = 0
                        longitude = 0
                    dict_jobs.append({
                        'intitule' : result['intitule'],
                        'link' : 'https://www.apec.fr/candidat/recherche-emploi.html/emploi/detail-offre/'+result['numeroOffre'],
                        'numero_offre' : result['numeroOffre'],
                        'lieuTexte' : result['lieuTexte'],
                        'texteOffre' : result['texteOffre'],
                        'nomCommercial' : result['nomCommercial'],
                        'score' : result['score'],
                        'salaireTexte' : result['salaireTexte'],
                        'typeContrat' : result['typeContrat'],
                        'datePublication' : result['datePublication'],
                        'dateValidation' : result['dateValidation'],
                        'latitude' : latitude,
                        'longitude' : longitude,
                        'contractDuration' : result['contractDuration'],
                    })
            
        ### retourne array
        return dict_jobs
 

## Webscrap

In [60]:
import pandas as pd
s_job = "data"
city = ""
code_dpt = ""
type_contract = ''

arr_jobs = []

print('please wait, search in progress...')

## apec
sjapec = scraping_jobs_apec(s_job, type_contract)
sjapec.set_code_dpt(code_dpt)
dict_tmp = sjapec.scrap_job()
if len(dict_tmp) > 0:
    arr_jobs += dict_tmp

df_apec = pd.DataFrame(arr_jobs)

please wait, search in progress...


In [61]:
df_apec.head(2)

,intitule,link,numero_offre,lieuTexte,texteOffre,nomCommercial,score,salaireTexte,typeContrat,datePublication,dateValidation,latitude,longitude,contractDuration
0,Data scientist/Data Analyst F/H,https://www.apec.fr/candidat/recherche-emploi....,172742239W,Vélizy-Villacoublay - 78,Nous cherchons : Bac +4 en école d'ingénieur ...,INNOVAI TEK,46.455513,45 - 55 k€ brut annuel,101888,2024-01-03T23:05:23.000+0000,2024-01-03T23:05:23.000+0000,"(48.7838513,)","(2.1973528,)",0
1,Data Analyst / Data engineer F/H,https://www.apec.fr/candidat/recherche-emploi....,172631521W,Paris 09 - 75,Auprès d'un de nos clients dans l'univers de l...,VR CONSEIL,44.474583,45 - 75 k€ brut annuel,101888,2023-12-14T15:48:38.000+0000,2023-12-14T15:48:38.000+0000,"(48.8771639,)","(2.3374635,)",0


# Traitement de la description

In [62]:
#récupérer la liste des ponctuations
import string
import ssl 
import nltk

ponctuations = list(string.punctuation)
print(ponctuations)


#outil pour procéder à la lemmatisation - attention à charger le cas échéant
#nltk.download()
from nltk.stem import WordNetLemmatizer
lem = WordNetLemmatizer()

#pour la tokénisation
from nltk.tokenize import word_tokenize

#liste des mots vides
from nltk.corpus import stopwords
mots_vides = stopwords.words("french")
print(mots_vides)

#********************************
#fonction pour nettoyage document (chaîne de caractères)
#le document revient sous la forme d'une liste de tokens
#********************************
def nettoyage_doc(doc_param):
    #passage en minuscule
    doc = doc_param.lower()
    #retrait des ponctuations
    doc = "".join([w for w in list(doc) if not w in ponctuations])
    #transformer le document en liste de termes par tokénisation
    doc = word_tokenize(doc)
    #lematisation de chaque terme
    doc = [lem.lemmatize(terme) for terme in doc]
    #retirer les stopwords
    doc = [w for w in doc if not w in mots_vides]
    #retirer les termes de moins de 3 caractères
    doc = [w for w in doc if len(w)>=1]
    #fin
    return doc

#************************************************************
#fonction pour nettoyage corpus
#attention, optionnellement les documents vides sont éliminés
#************************************************************
def nettoyage_corpus(corpus,vire_vide=True):
    #output
    output = [nettoyage_doc(doc) for doc in corpus if ((len(doc) > 0) or (vire_vide == False))]
    return output


#corpus après pré-traitement, sous forme de listes de liste de tokens
ex_search_results = ex_search.get('resultats', [])  # Récupérer la liste des résultats
# Récupérer les descriptions
descriptions_offres = [offre.get('description', '') for offre in ex_search_results]
descriptions_offres
desc = nettoyage_corpus(descriptions_offres)

['!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~']
['au', 'aux', 'avec', 'ce', 'ces', 'dans', 'de', 'des', 'du', 'elle', 'en', 'et', 'eux', 'il', 'ils', 'je', 'la', 'le', 'les', 'leur', 'lui', 'ma', 'mais', 'me', 'même', 'mes', 'moi', 'mon', 'ne', 'nos', 'notre', 'nous', 'on', 'ou', 'par', 'pas', 'pour', 'qu', 'que', 'qui', 'sa', 'se', 'ses', 'son', 'sur', 'ta', 'te', 'tes', 'toi', 'ton', 'tu', 'un', 'une', 'vos', 'votre', 'vous', 'c', 'd', 'j', 'l', 'à', 'm', 'n', 's', 't', 'y', 'été', 'étée', 'étées', 'étés', 'étant', 'étante', 'étants', 'étantes', 'suis', 'es', 'est', 'sommes', 'êtes', 'sont', 'serai', 'seras', 'sera', 'serons', 'serez', 'seront', 'serais', 'serait', 'serions', 'seriez', 'seraient', 'étais', 'était', 'étions', 'étiez', 'étaient', 'fus', 'fut', 'fûmes', 'fûtes', 'furent', 'sois', 'soit', 'soyons', 'soyez', 'soient', 'fusse', 'fusses', 'fût', 'fussions', 'fus

# CREATION DES TABLES

In [63]:
import sqlite3

In [64]:
conn = sqlite3.connect('job_mining.db')

In [65]:
cursor = conn.cursor()
cursor.execute('''
CREATE TABLE IF NOT EXISTS LieuTravail_Dimension (
    id TEXT PRIMARY KEY,
    libelle TEXT,
    latitude REAL,
    longitude REAL,
    codePostal TEXT,
    commune TEXT
)
''')



In [66]:
cursor.execute('''
CREATE TABLE IF NOT EXISTS Entreprise_Dimension (
    id TEXT PRIMARY KEY,
    nom TEXT,
    adaptee INTEGER
)
''')


In [67]:
cursor.execute('''
CREATE TABLE IF NOT EXISTS Contrat_Dimension (
    id TEXT PRIMARY KEY,
    typeContrat TEXT,
    typeContratLibelle TEXT,
    natureContrat TEXT,
    dureeTravailLibelle TEXT,
    dureeTravailLibelleConverti TEXT,
    alternance INTEGER
)
''')

In [68]:
cursor.execute('''
    CREATE TABLE IF NOT EXISTS OrigineOffre_Dimension (
        id TEXT PRIMARY KEY,
        origine TEXT,
        urlOrigine TEXT
    )
''')

In [69]:
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Qualification_Dimension (
        id TEXT PRIMARY KEY,
        qualification TEXT
    )
''')

In [70]:
# Création de la table de faits (OffresEmploi_Faits)
cursor.execute('''
    CREATE TABLE IF NOT EXISTS OffresEmploi_Faits (
        id TEXT PRIMARY KEY,
        poste TEXT,
        dateCreation TEXT,
        dateActualisation TEXT,
        description TEXT,
        nombrePostes INTEGER,
        salaireLibelle TEXT,
        lieuTravailId TEXT REFERENCES LieuTravail_Dimension(id),
        entrepriseId TEXT REFERENCES Entreprise_Dimension(id),
        contratId TEXT REFERENCES Contrat_Dimension(id),
        qualificationId TEXT REFERENCES Qualification_Dimension(id),
        origineOffreId TEXT REFERENCES OrigineOffre_Dimension(id)
    )
''')

In [71]:
# Commit et fermeture de la connexion
conn.commit()
conn.close()

# Pre-traitement des tables

## Chargement table  --- LieuTravail_Dimension

### POLE EMPLOI

In [72]:
data = {
    'id': [],
    'libelle': [],
    'latitude': [],
    'longitude': [],
    'codePostal': [],
    'commune': []
}

for job_posting in ex_search.get('resultats', []):
    lieu_travail = job_posting.get('lieuTravail', {})
    data['id'].append(lieu_travail.get('libelle', ''))
    data['libelle'].append(lieu_travail.get('libelle', ''))
    data['latitude'].append(lieu_travail.get('latitude', ''))
    data['longitude'].append(lieu_travail.get('longitude', ''))
    data['codePostal'].append(lieu_travail.get('codePostal', ''))
    data['commune'].append(lieu_travail.get('commune', ''))

# Create a DataFrame
df_lieu = pd.DataFrame(data)

### APEC

In [73]:
# Création du DataFrame df_lieu_apec
df_lieu_apec = pd.DataFrame({
    'id': range(1, len(df_apec) + 1),
    'libelle': df_apec['lieuTexte'],
    'latitude': df_apec['latitude'],
    'longitude': df_apec['longitude'],
    'codePostal': df_apec['lieuTexte'].str[-2:],
    'commune': ''
})
df_lieu_apec['latitude'] = df_lieu_apec['latitude'].apply(lambda x: x[0] if isinstance(x, tuple) else x)
df_lieu_apec['longitude'] = df_lieu_apec['longitude'].apply(lambda x: x[0] if isinstance(x, tuple) else x)

### JOINTURE

In [74]:
df_lieu = pd.concat([df_lieu, df_lieu_apec], ignore_index=True)
df_lieu = df_lieu.drop_duplicates(subset=['libelle'])
df_lieu = df_lieu.reset_index(drop=True)
df_lieu['id'] = df_lieu.index + 1
df_lieu.head()

,id,libelle,latitude,longitude,codePostal,commune
0,1,31 - TOULOUSE,43.604082,1.433805,31000,31555
1,2,75 - PARIS 11,48.860066,2.381835,75011,75111
2,3,92 - SURESNES,48.870987,2.218099,92150,92073
3,4,75 - PARIS 09,48.877098,2.337887,75009,75109
4,5,33 - BORDEAUX,44.851895,-0.587877,33000,33063


## Chargement de la table --- Entreprise_Dimension

### POLE EMPLOI

In [75]:
data_entreprises = {
    'id': [],
    'nom': [],
    'adaptee': []
}

for job_posting in ex_search.get('resultats', []):
    entreprise = job_posting.get('entreprise', {})
    
    data_entreprises['id'].append(entreprise.get('nom', ''))
    data_entreprises['nom'].append(entreprise.get('nom', ''))
    data_entreprises['adaptee'].append(entreprise.get('entrepriseAdaptee', ''))
df_entreprises = pd.DataFrame(data_entreprises)

### APEC

In [76]:
# Création du DataFrame df_entreprise_apec
df_entreprise_apec = pd.DataFrame({
    'id': range(1, len(df_apec) + 1),
    'nom': df_apec['nomCommercial'],
    'adaptee': ''
})

### JOINTURE

In [77]:
df_entreprises = pd.concat([df_entreprises, df_entreprise_apec], ignore_index=True)
df_entreprises = df_entreprises.drop_duplicates(subset=['nom'])
df_entreprises = df_entreprises.reset_index(drop=True)
df_entreprises['id'] = df_entreprises.index + 1
df_entreprises.head()


,id,nom,adaptee
0,1,CS GROUP - FRANCE,False
1,2,THE PURE PROJECT,False
2,3,SPVIE,False
3,4,,False
4,5,DATAYETT CONSULTING,False


## Chargement de la table --- Contrat_Dimension

### POLE EMPLOI

In [78]:
data_contrat = {
    'id': [],
    'typeContrat': [],
    'typeContratLibelle': [],
    'natureContrat': [],
    'dureeTravailLibelle': [],
    'dureeTravailLibelleConverti': [],
    'alternance': []
}

for job_posting in ex_search.get('resultats', []):
    contrat = job_posting
    
    data_contrat['id'].append(contrat.get('id', ''))
    data_contrat['typeContrat'].append(contrat.get('typeContrat', ''))
    data_contrat['typeContratLibelle'].append(contrat.get('typeContratLibelle', ''))
    data_contrat['natureContrat'].append(contrat.get('natureContrat', ''))
    data_contrat['dureeTravailLibelle'].append(contrat.get('dureeTravailLibelle', ''))
    data_contrat['dureeTravailLibelleConverti'].append(contrat.get('dureeTravailLibelleConverti', ''))
    data_contrat['alternance'].append(contrat.get('alternance', ''))

# Create a DataFrame for contrat
df_contrat = pd.DataFrame(data_contrat)

### APEC

In [79]:
# Création du DataFrame df_contrat_apec
df_contrat_apec = pd.DataFrame({
    'id': range(1, len(df_apec) + 1),
    'typeContrat': df_apec['typeContrat'],
    'typeContratLibelle': '',
    'natureContrat': '',
    'dureeTravailLibelle': '',
    'dureeTravailLibelleConverti': '',
    'alternance': ''
})
# Mapping codes to contract types
contract_mapping = {
    101888: 'CDI',
    101887: 'CDD',
    101889: 'Interim'
}
# Applying the mapping to create a new column 'contract_type'
df_contrat_apec['typeContrat'] = df_contrat_apec['typeContrat'].map(contract_mapping)

### JOINTURE 

In [80]:
df_contrat = pd.concat([df_contrat, df_contrat_apec], ignore_index=True)
df_contrat['id'] = df_contrat.index + 1
df_contrat.head()


,id,typeContrat,typeContratLibelle,natureContrat,dureeTravailLibelle,dureeTravailLibelleConverti,alternance
0,1,CDI,Contrat à durée indéterminée,Contrat travail,36H40 Travail en journée,Temps plein,False
1,2,CDI,Contrat à durée indéterminée,Contrat travail,37H05 Travail en journée,Temps plein,False
2,3,CDI,Contrat à durée indéterminée,Contrat travail,39H Travail en journée,Temps plein,False
3,4,CDD,Contrat à durée déterminée - 12 Mois,Contrat apprentissage,35H Travail en journée,Temps plein,True
4,5,CDI,Contrat à durée indéterminée,CDI de chantier ou d'opération,35H Travail en journée,Temps plein,False


## Chargement de la table --- OrigineOffre_Dimension

### POLE EMPLOI

In [81]:
data_origine = {
    'id': [],
    'origine': [],
    'urlOrigine': []
}

for job_posting in ex_search.get('resultats', []):
    origine_offre = job_posting.get('origineOffre', {})
    
    data_origine['id'].append(origine_offre.get('id', ''))
    data_origine['origine'].append(origine_offre.get('origine', ''))
    data_origine['urlOrigine'].append(origine_offre.get('urlOrigine', ''))

# Create a DataFrame for origine
df_origine = pd.DataFrame(data_origine)

### APEC

In [82]:
df_origine_apec = pd.DataFrame({
    'id': range(3, len(df_apec) + 1),
    'origine': 'APEC',
    'adaptee': ''
})

### JOINTURE

In [83]:
df_origine = pd.concat([df_origine, df_origine_apec], ignore_index=True)
df_origine['id'] = df_origine.index + 1
df_origine.head()

,id,origine,urlOrigine,adaptee
0,1,1,https://candidat.pole-emploi.fr/offres/recherc...,NaN
1,2,1,https://candidat.pole-emploi.fr/offres/recherc...,NaN
2,3,1,https://candidat.pole-emploi.fr/offres/recherc...,NaN
3,4,1,https://candidat.pole-emploi.fr/offres/recherc...,NaN
4,5,1,https://candidat.pole-emploi.fr/offres/recherc...,NaN


## Chargement de la table --- Qualification_Dimension

### POLE EMPLOI

In [84]:
data_qualification = {
    'id': [],
    'qualification': []
}
for job_posting in ex_search.get('resultats', []):
    data_qualification['id'].append(job_posting.get('id', ''))
    data_qualification['qualification'].append(job_posting.get('qualificationLibelle', ''))

# Create a DataFrame for qualification
df_qualification = pd.DataFrame(data_qualification)

### APEC

In [85]:
# Création du DataFrame df_qualification_apec
df_qualification_apec = pd.DataFrame({
    'id': range(1, len(df_apec) + 1),
    'qualification': ''
})

### JOINTURE

In [86]:
df_qualification = pd.concat([df_qualification, df_qualification_apec], ignore_index=True)
df_qualification = df_qualification.drop_duplicates(subset=['qualification'])
df_qualification = df_qualification.reset_index(drop=True)
df_qualification['id'] = df_qualification.index + 1
df_qualification


,id,qualification
0,1,Cadre
1,2,Agent de maîtrise
2,3,Employé qualifié
3,4,Technicien
4,5,


## Chargement de la table --- OffresEmploi_Faits

### POLE EMPLOI

In [87]:
import sqlite3
import pandas as pd

# Extracting data for all job postings
data = {
    'id': [],
    'poste':[],
    'dateCreation': [],
    'dateActualisation': [],
    'description': [],
    'nombrePostes': [],
    'salaireLibelle': [],
    'lieuTravailId': [],
    'entrepriseId': [],
    'contratId': [],
    'qualificationId': [],
    'origineOffreId': []
}

for job_posting in ex_search.get('resultats', []):
    data['id'].append(job_posting.get('id', ''))
    data['poste'].append(job_posting.get('intitule', ''))
    data['dateCreation'].append(job_posting.get('dateCreation', ''))
    data['dateActualisation'].append(job_posting.get('dateActualisation', ''))
    data['nombrePostes'].append(job_posting.get('nombrePostes', ''))
    data['salaireLibelle'].append(job_posting.get('salaire', {}).get('libelle', ''))
    data['lieuTravailId'].append(job_posting.get('lieuTravail', {}).get('libelle', ''))
    data['entrepriseId'].append(job_posting.get('entreprise', {}).get('nom', ''))
    data['contratId'].append(job_posting.get('typeContratLibelle', ''))
    data['qualificationId'].append(job_posting.get('qualificationLibelle', ''))
    data['origineOffreId'].append(job_posting.get('origineOffre', {}).get('origine', ''))


for i in range(0,len(ex_search.get('resultats', []))):
     data['description'].append(desc[i])


# Create a DataFrame
df_offreF = pd.DataFrame(data)

# Convertir la colonne 'description' de liste à chaîne de caractères
df_offreF['description'] = df_offreF['description'].apply(lambda x: ' '.join(x))



### APEC

In [88]:
df_apec['typeContrat'] = df_apec['typeContrat'].map(contract_mapping)
df_offre_apec = pd.DataFrame({
    'id': df_apec['numero_offre'],
    'poste': df_apec['intitule'],
    'dateCreation': df_apec['numero_offre'],  # You may need to adjust this column based on your data
    'dateActualisation': df_apec['numero_offre'],  # You may need to adjust this column based on your data
    'description': df_apec['numero_offre'],  # You may need to adjust this column based on your data
    'nombrePostes': 1,
    'salaireLibelle': df_apec['salaireTexte'],
    'lieuTravailId' : df_apec['lieuTexte'],
    'entrepriseId': df_apec['nomCommercial'],
    'contratId': df_apec['typeContrat'],
    'qualificationId': "",  # You may need to adjust this value based on your data
    'origineOffreId': 3
})

### Jointure

In [89]:
df_offreF = pd.concat([df_offreF, df_offre_apec], ignore_index=True)

In [90]:
df_offreF.head()

,id,poste,dateCreation,dateActualisation,description,nombrePostes,salaireLibelle,lieuTravailId,entrepriseId,contratId,qualificationId,origineOffreId
0,167PCTC,Ingénieur Data modeler et/ou TechData - (H/F),2024-01-11T17:27:45.000Z,2024-01-11T17:27:47.000Z,recrutons ingénieur data modeler etou tech dat...,2,"Annuel de 38000,00 Euros à 45000,00 Euros sur ...",31 - TOULOUSE,CS GROUP - FRANCE,Contrat à durée indéterminée,Cadre,1
1,167PCCB,Data Product Manager (H/F),2024-01-11T17:11:32.000Z,2024-01-11T17:11:33.000Z,data product manager pilotera lautomatisation ...,1,,75 - PARIS 11,THE PURE PROJECT,Contrat à durée indéterminée,Agent de maîtrise,1
2,167NXWN,Data Manager (H/F),2024-01-11T16:03:58.000Z,2024-01-11T16:04:00.000Z,sein léquipe rôle serum construire dactualiser...,1,,92 - SURESNES,SPVIE,Contrat à durée indéterminée,Cadre,1
3,167MLTP,Account manager/data (H/F),2024-01-10T16:40:55.000Z,2024-01-11T14:15:59.000Z,recherchons account manager data accompagner l...,1,,75 - PARIS 09,,Contrat à durée déterminée - 12 Mois,Employé qualifié,1
4,167MKYX,Data Engineer (3 ans d'exp min) (H/F),2024-01-10T16:27:22.000Z,2024-01-11T10:46:19.000Z,chez datayett entreprise jeune dynamique spéci...,1,"Mensuel de 40000,00 Euros à 50000,00 Euros sur...",33 - BORDEAUX,DATAYETT CONSULTING,Contrat à durée indéterminée,Cadre,1


### Modification des champs

In [91]:
df_lieu_selection = df_lieu[['id', 'libelle']]
df_entreprises_selection = df_entreprises[['id', 'nom']]
df_contrat_selection = df_contrat[['id', 'typeContratLibelle']]
df_origine_selection = df_origine[['id', 'origine']]
df_qualification_selection = df_qualification[['id', 'qualification']]

#LIEU
df_offreF = pd.merge(df_offreF, df_lieu_selection, how='left', left_on='lieuTravailId', right_on='libelle')
df_offreF = df_offreF.drop(columns=['lieuTravailId','libelle'])
df_offreF = df_offreF.rename(columns={'id_y': 'lieuTravailId'})
df_offreF = df_offreF.rename(columns={'id_x': 'id'})

#ENTREPRISE
df_offreF = pd.merge(df_offreF, df_entreprises_selection, how='left', left_on='entrepriseId', right_on='nom')
df_offreF = df_offreF.drop(columns=['entrepriseId','nom'])
df_offreF = df_offreF.rename(columns={'id_y': 'entrepriseId'})
df_offreF = df_offreF.rename(columns={'id_x': 'id'})

#CONTRAT
df_offreF = pd.merge(df_offreF, df_contrat_selection, how='left', left_on='contratId', right_on='typeContratLibelle')
df_offreF = df_offreF.drop(columns=['contratId','typeContratLibelle'])
df_offreF = df_offreF.rename(columns={'id_y': 'contratId'})
df_offreF = df_offreF.rename(columns={'id_x': 'id'})

#ORIGINE
df_offreF['origineOffreId'] = df_offreF['origineOffreId'].astype(str)
df_offreF = pd.merge(df_offreF, df_origine_selection, how='left', left_on='origineOffreId', right_on='origine')
df_offreF = df_offreF.drop(columns=['origineOffreId','origine'])
df_offreF = df_offreF.rename(columns={'id_y': 'origineOffreId'})
df_offreF = df_offreF.rename(columns={'id_x': 'id'})

#QUALIFICATION
df_offreF = pd.merge(df_offreF, df_qualification_selection, how='left', left_on='qualificationId', right_on='qualification')
df_offreF = df_offreF.drop(columns=['qualificationId','qualification'])
df_offreF = df_offreF.rename(columns={'id_y': 'qualificationId'})
df_offreF = df_offreF.rename(columns={'id_x': 'id'})

# Liste des colonnes dans le nouvel ordre souhaité
nouvel_ordre_colonnes = [
    'id',
    'poste',
    'dateCreation',
    'dateActualisation',
    'description',
    'nombrePostes',
    'salaireLibelle',
    'lieuTravailId',
    'entrepriseId',
    'contratId',
    'qualificationId',
    'origineOffreId'
]
df_offreF = df_offreF[nouvel_ordre_colonnes]

In [92]:
df_offreF.head(2)

,id,poste,dateCreation,dateActualisation,description,nombrePostes,salaireLibelle,lieuTravailId,entrepriseId,contratId,qualificationId,origineOffreId
0,167PCTC,Ingénieur Data modeler et/ou TechData - (H/F),2024-01-11T17:27:45.000Z,2024-01-11T17:27:47.000Z,recrutons ingénieur data modeler etou tech dat...,2,"Annuel de 38000,00 Euros à 45000,00 Euros sur ...",1,1,1.0,1,1.0
1,167PCTC,Ingénieur Data modeler et/ou TechData - (H/F),2024-01-11T17:27:45.000Z,2024-01-11T17:27:47.000Z,recrutons ingénieur data modeler etou tech dat...,2,"Annuel de 38000,00 Euros à 45000,00 Euros sur ...",1,1,1.0,1,2.0


In [47]:
df_offreF['contratId'].unique()

array([  1.,   2.,   3.,   5.,   6.,   7.,   8.,  10.,  11.,  12.,  14.,
        16.,  18.,  20.,  21.,  22.,  23.,  24.,  25.,  27.,  28.,  29.,
        30.,  31.,  32.,  33.,  34.,  35.,  36.,  37.,  38.,  39.,  40.,
        41.,  42.,  43.,  44.,  45.,  46.,  47.,  48.,  50.,  51.,  52.,
        54.,  55.,  56.,  57.,  58.,  59.,  60.,  61.,  62.,  63.,  65.,
        66.,  67.,  68.,  69.,  70.,  71.,  72.,  73.,  76.,  77.,  78.,
        79.,  80.,  81.,  83.,  84.,  85.,  86.,  87.,  89.,  90.,  91.,
        92.,  93.,  94.,  95.,  96.,  97.,  98.,  99., 100., 101., 102.,
       104., 105., 106., 107., 108., 109., 110., 111., 112., 113., 114.,
       115., 116., 117., 118., 119., 120., 121., 122., 123., 124., 125.,
       127., 128., 129., 130., 131., 132., 133., 134., 135., 136., 137.,
       138., 139., 142., 143., 144., 145., 149., 150.,   4.,  13.,  19.,
        49., 140.,   9., 126.,  15.,  64.,  82.,  17.,  26.,  53., 146.,
        74.,  75.,  88., 103., 141., 147., 148.,  n

In [93]:
# Appliquer les fonctions de nettoyage à la variable "poste" de df_OffreF
df_offreF['poste'] = df_offreF['poste'].apply(lambda x: ' '.join(nettoyage_doc(x)))

# Afficher le DataFrame après le nettoyage
print(df_offreF)

                 id                                    poste  \
0           167PCTC  ingénieur data modeler etou techdata hf   
1           167PCTC  ingénieur data modeler etou techdata hf   
2           167PCTC  ingénieur data modeler etou techdata hf   
3           167PCTC  ingénieur data modeler etou techdata hf   
4           167PCTC  ingénieur data modeler etou techdata hf   
...             ...                                      ...   
2504455  172757109W                          data analyst fh   
2504456  172714664W                          data analyst fh   
2504457  172641526W                          data analyst fh   
2504458  172765814W                          data analyst fh   
2504459  172735161W                        data analyst ° fh   

                     dateCreation         dateActualisation  \
0        2024-01-11T17:27:45.000Z  2024-01-11T17:27:47.000Z   
1        2024-01-11T17:27:45.000Z  2024-01-11T17:27:47.000Z   
2        2024-01-11T17:27:45.000Z  2024-01

In [101]:
resultats=df_offreF[df_offreF['poste'].str.contains('data analyst', case=False)]

resultats_sans_doublons = resultats.drop_duplicates(subset='id')

In [110]:
resultats_sans_doublons.shape

(10, 12)

In [104]:
ids_specifiques = ['167MHRY', '167KSDF', '167KKHH', '167FYQD', '167DGHK', '167CZKY', '167CNBM', '166ZRRV', '166ZPMT', '166ZMHH']

# Utilisez la méthode .isin() pour filtrer les lignes
resultats = df_offreF[df_offreF['id'].isin(ids_specifiques)]

In [106]:
resultats_sans_doublons = resultats.drop_duplicates(subset='id')

In [111]:
resultats_sans_doublons.shape

(10, 12)

# CHARGEMENT DES TABLES

In [94]:
import sqlite3
import pandas as pd
# Connexion à la base de données SQLite
conn = sqlite3.connect('job_mining.db')
###
# Chargement des tables
df_lieu.to_sql('LieuTravail_Dimension', conn, if_exists='replace', index=False)
df_entreprises.to_sql('Entreprise_Dimension', conn, if_exists='replace', index=False)
df_contrat.to_sql('Contrat_Dimension', conn, if_exists='replace', index=False)
df_origine.to_sql('OrigineOffre_Dimension', conn, if_exists='replace', index=False)
df_qualification.to_sql('Qualification_Dimension', conn, if_exists='replace', index=False)
df_offreF.to_sql('OffresEmploi_Faits', conn, if_exists='replace', index=False)

# Fermer la connexion
conn.close()


In [96]:
conn = sqlite3.connect('job_mining.db')
cursor = conn.cursor()
cursor.execute("SELECT * FROM OffresEmploi_Faits LIMIT 5")

# Fetch all rows
rows = cursor.fetchall()

# Display the results
for row in rows:
    print(row)


conn.close()

('167PCTC', 'ingénieur data modeler etou techdata hf', '2024-01-11T17:27:45.000Z', '2024-01-11T17:27:47.000Z', 'recrutons ingénieur data modeler etou tech data rejoindre business unit industrie sein business line data intelligence accompagne no client leurs problématiques associées transformation digitale no offres déclinent autour data intelligence maintenance prédictive client tant service responsable définition gouvernance données a mission globale spécifier modéliser données technique leurs affichages aussi charge processus lexécution opérations gouvernance données cette mission fortement liée stratégie digitalisation client cadre mission intégrée léquipe tech data modélisation données afin analyser besoins métiers décrire modéliser objets données métiers catalogue dobjets données métiers fournir modèles données projets concevoir spécifications données métiers projets liés données valider nouvelles version production données rapport spécifications apporter support fonctionnel équip